In [1]:
import numpy as np
import operator

In [2]:
#PH = "BC331D000103008066390ACC"
PH = "BC11021000E1008120CC" #telegram in hex
Controlf = "B0B8B4BC9C"     #control field in hex
AC = "0CC0CC"               #acknoledgement field in hex

In [3]:
def hextobin(h):
    return bin(int(h, 16))[2:].zfill(len(h) * 4)

In [4]:
def bitlength(n):
    return len(n)

In [5]:
PB = hextobin(PH)           #telegram in binary
PB

'10111100000100010000001000010000000000001110000100000000100000010010000011001100'

In [6]:
NB = bitlength(PB)
NB

80

In [7]:
def hextobytearray(s):
    byte_array = bytearray.fromhex(s)
    return byte_array
hextobytearray(PH)

bytearray(b'\xbc\x11\x02\x10\x00\xe1\x00\x81 \xcc')

In [8]:
ACK = hextobytearray(AC)    #ack field to byte array

In [9]:
def bitstringtobytes(s):
    return int(s, 2).to_bytes(len(s) // 8, byteorder='big')

print(bitstringtobytes(PB))


b'\xbc\x11\x02\x10\x00\xe1\x00\x81 \xcc'


In [10]:
def bytearrayrange(n):
    bytes = range(n) 
    byte_array = bytearray(bytes)
    return byte_array
Lf = bytearrayrange(16)

In [11]:
PBlast = hextobytearray(PH)[-1:]

In [12]:
if (72 <= NB <= 192):
    if (hextobytearray(PH)[:1] in hextobytearray(Controlf)):
        if(PBlast in ACK):
            print("yes")


yes


In [13]:
CHb =  hextobytearray(PH)[-3:-2]     #checksum byte array
CHb

bytearray(b'\x81')

In [14]:
PHbch =  hextobytearray(PH)[:-3]    #telegram before the checksum in byte array
PHbch

bytearray(b'\xbc\x11\x02\x10\x00\xe1\x00')

In [15]:
x2 = bin(int.from_bytes(bitstringtobytes(PB), byteorder="big")).strip('0b') #convert bytestring to bits string
digits = [int(x) for x in x2]

In [16]:
srcAddr = hextobytearray(PH)[1:3]  #source address in byte array
srcAddr


bytearray(b'\x11\x02')

In [17]:
data = hextobytearray(PH)
data

bytearray(b'\xbc\x11\x02\x10\x00\xe1\x00\x81 \xcc')

In [18]:
def bool2int(x):
    y = 0
    for i,j in enumerate(x):
        y += j<<i
    return y

In [19]:
def sourceAddress(data):
    x = np.frombuffer(bitstringtobytes(data), dtype=np.uint8)
    newarr = np.unpackbits(x).reshape(-1, 4)
    add = newarr[2:6]
    area = np.array([add[0]])
    line = np.array([add[1]])
    area_line = np.concatenate((area, line))
    participant = np.concatenate((add[2],add[3]))
    c = [bool2int(i[::-1]) for i in  area_line]  
    res = int("".join(str(x) for x in participant), 2)

    c.append(res)                                  #return c for address in array format

    return  "%d.%d.%d"%(c[0], c[1], res)

src = sourceAddress(PB)

print(src)

1.1.2


In [20]:
def destinationAddress(data):
    x = np.frombuffer(bitstringtobytes(data), dtype=np.uint8)
    newarr = np.unpackbits(x).reshape(-1, 8)

    check = newarr[5][0]            #check if it is a physical or group address
    check3 = np.array(newarr[3])
    check4 = np.array(newarr[4])

    if(check == 0):   #physical address

        address = np.concatenate((check3,check4))
        new = address.reshape(-1, 4)
        main = np.array([new[0]])
        middle = np.array([new[1]])
        main_middle = np.concatenate((main, middle))
        subgroup = np.concatenate((new[2],new[3]))
        c = [bool2int(i[::-1]) for i in  main_middle]   
        res = int("".join(str(x) for x in subgroup), 2)
        c.append(res) 

        return  "%d/%d/%d"%(c[0], c[1], res)

    else:               #group address
        address = np.concatenate((check3,check4))
        main =  int("".join(str(x) for x in address[:5]), 2)
        middle = int("".join(str(x) for x in address[5:8]), 2)
        sub =  int("".join(str(x) for x in address[8:]), 2)

        return "%d/%d/%d"%(main, middle, sub)

des = destinationAddress(PB)

print(des)

2/0/0


In [21]:
def routingCounter(data):
    x = np.frombuffer(bitstringtobytes(data), dtype=np.uint8)
    newarr = np.unpackbits(x).reshape(-1, 8)[5]
    new = newarr.reshape(-1, 4)[0][1:]
    counter =  int("".join(str(x) for x in new), 2)
    return counter

route = routingCounter(PB)
print(route)

6


In [22]:
def payloadLength(data):
    x = np.frombuffer(bitstringtobytes(data), dtype=np.uint8)
    newarr = np.unpackbits(x).reshape(-1, 8)[5]
    new = newarr.reshape(-1, 4)[1]
    payload =  int("".join(str(x) for x in new), 2)
    return payload

load = payloadLength(PB)
print(load)

1


In [23]:
def crc16(data: bytes):
    xor_in = 0x0000  # initial value
    xor_out = 0x0000  # final XOR value
    poly = 0x8005  # generator polinom (normal form)

    reg = xor_in
    for octet in data:
        # reflect in
        for i in range(8):
            topbit = reg & 0x8000
            if octet & (0x80 >> i):
                topbit ^= 0x8000
            reg <<= 1
            if topbit:
                reg ^= poly
        reg &= 0xFFFF
        # reflect out
    return reg ^ xor_out
crc = crc16(bitstringtobytes(PB))
crc

15624

In [24]:
def calculate_checksum(payload):
	magicValue = 0x55
	mask = 0xFF

	checksum = 0
	for element in payload:
		checksum = operator.add(checksum,element)
	checksum =  operator.xor(checksum,magicValue)
	return checksum & mask

In [25]:
calculate_checksum(data)

120

In [26]:
# Declaring byte value  BCE0361C12A502008009C4
byte_val = b'\xbc'
#byte = b'\x36\x1C\x12\xA5\x02\x00\x80\x09\xC4'
byte = b'\x1C\x12'

In [27]:
# Converting to int
int_val = int.from_bytes(byte_val, "big")
int_va1 = int.from_bytes(byte, "big")

In [28]:
# printing int equivalent
print(int_val,int_va1)

188 7186
